<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#P4-catalog-stats" data-toc-modified-id="P4-catalog-stats-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>P4 catalog stats</a></span></li></ul></div>

In [1]:
%load_ext lab_black

In [2]:
import seaborn as sns

sns.set_context("notebook")

In [3]:
obsids = pd.read_csv("./image_names.csv", header=None, names=["obsid"])

In [4]:
obsids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 1 columns):
obsid    462 non-null object
dtypes: object(1)
memory usage: 3.7+ KB


In [5]:
from planetarypy.pdstools import indices

In [6]:
edrindex = indices.indexdb.get("mro.hirise.edr")

In [7]:
edrindex.local_hdf_path

PosixPath('/Users/klay6683/Dropbox/data/planetarypy/indices/mro/hirise/EDRCUMINDEX.hdf')

In [8]:
meta = pd.read_hdf(edrindex.local_hdf_path)

In [9]:
meta[meta.OBSERVATION_ID.str.startswith("ESP")].OBSERVATION_ID.max()

'ESP_061798_2640'

In [10]:
meta.columns

Index(['VOLUME_ID', 'FILE_NAME_SPECIFICATION', 'INSTRUMENT_HOST_ID',
       'INSTRUMENT_ID', 'OBSERVATION_ID', 'PRODUCT_ID', 'PRODUCT_VERSION_ID',
       'HICAL_VERSION', 'TARGET_NAME', 'ORBIT_NUMBER', 'MISSION_PHASE_NAME',
       'RATIONALE_DESC', 'OBSERVATION_START_TIME', 'OBSERVATION_START_COUNT',
       'START_TIME', 'SPACECRAFT_CLOCK_START_COUNT', 'STOP_TIME',
       'SPACECRAFT_CLOCK_STOP_COUNT', 'CCD_NAME', 'CHANNEL_NUMBER',
       'FILTER_NAME', 'SCAN_EXPOSURE_DURATION', 'DELTA_LINE_TIME_COUNT',
       'BINNING', 'TDI', 'TRIM_LINES', 'FOCUS_POSITION_COUNT',
       'FELICS_COMPRESSION_FLAG', 'STIMULATION_LAMP_FLAG_RED',
       'STIMULATION_LAMP_FLAG_BLUEGREEN', 'STIMULATION_LAMP_FLAG_NEARINFRARED',
       'LOOKUP_TABLE_TYPE', 'LOOKUP_TABLE_MINIMUM', 'LOOKUP_TABLE_MAXIMUM',
       'LOOKUP_TABLE_MEDIAN', 'LOOKUP_TABLE_K_VALUE', 'LOOKUP_TABLE_NUMBER',
       'ADC_CONVERSION_SETTINGS_1', 'ADC_CONVERSION_SETTINGS_2',
       'FPA_POSITIVE_Y_TEMPERATURE', 'FPA_NEGATIVE_Y_TEMPERATURE',


In [11]:
meta.drop_duplicates(subset="OBSERVATION_ID", inplace=True)

In [12]:
meta_cols = [
    "OBSERVATION_ID",
    "IMAGE_CENTER_LATITUDE",
    "IMAGE_CENTER_LONGITUDE",
]

In [13]:
obsids

,obsid
0,ESP_012079_0945
1,ESP_011697_0980
2,ESP_021455_0935
3,ESP_012008_0975
4,ESP_012436_0980
...,...
457,ESP_046770_0950
458,ESP_046769_0950
459,ESP_046862_0950
460,ESP_040216_0950


In [21]:
df = obsids.merge(
    meta[meta_cols], left_on="obsid", right_on="OBSERVATION_ID", how="inner"
).drop("OBSERVATION_ID", axis=1)
df.head()

,obsid,IMAGE_CENTER_LATITUDE,IMAGE_CENTER_LONGITUDE
0,ESP_012079_0945,-85.4074,103.9700
1,ESP_011697_0980,-81.8089,76.1636
2,ESP_021455_0935,-86.3962,99.1061
3,ESP_012008_0975,-82.1867,225.2310
4,ESP_012436_0980,-81.9405,60.4513


In [34]:
df = meta[meta_cols].rename({"OBSERVATION_ID": "obsid"}, axis=1)

In [35]:
df = df[(df.obsid.str.startswith("PSP")) | (df.obsid.str.startswith("ESP"))]

In [36]:
df.head()

,obsid,IMAGE_CENTER_LATITUDE,IMAGE_CENTER_LONGITUDE
2827,PSP_001330_1395,-40.1045,132.2780
2855,PSP_001331_2260,45.6201,93.6834
2871,PSP_001332_2620,81.8166,48.7460
2899,PSP_001333_2485,68.1677,33.5881
2915,PSP_001334_2215,40.9504,12.0690


In [17]:
%matplotlib widget

In [37]:
df.columns = ["obsid", "lat", "lon"]

In [38]:
df.shape

(62936, 3)

In [46]:
region_names = pd.read_csv(
    "/Users/klay6683/Dropbox/src/planet4/planet4/data/region_names.csv"
)
region_names.columns = ["name", "lat", "lon"]

In [47]:
region_names.shape

(27, 3)

In [48]:
region_names.head()

,name,lat,lon
0,Albany,-81.930,60.400
1,Atka,-86.980,169.700
2,Bilbao,-87.008,127.273
3,Binghamton,-73.530,339.500
4,BuenosAires,-81.901,4.750


In [49]:
from scipy.spatial.distance import cdist, pdist

In [50]:
from geopy.distance import distance
from geopy import Point


def create_point(row):
    return Point(latitude=row.lat, longitude=row.lon)

In [51]:
region_names["point"] = region_names[["lat", "lon"]].apply(create_point, axis=1)

In [52]:
df.loc[df.lat < -9997, "lat"] = np.nan

In [53]:
df.loc[df.lon < -9997, "lon"] = np.nan

In [57]:
df.shape

(62936, 3)

In [58]:
df = df.dropna(how="all", subset=["lat", "lon"])

In [59]:
df["point"] = df[["lat", "lon"]].apply(create_point, axis=1)

In [60]:
distance(region_names.point.iloc[0], df.point.iloc[0]).kilometers

5331.443072200442

In [61]:
Y = cdist(
    df[["point"]], region_names[["point"]], lambda x, y: distance(x, y).kilometers,
)

In [62]:
df["geopy_names"] = region_names.iloc[Y.argmin(axis=1)].name.values

In [63]:
df["minimal_distance"] = Y.min(axis=1)

In [64]:
merged = (
    df.drop("point", axis=1)
    .merge(
        region_names.drop("point", axis=1),
        left_on="geopy_names",
        right_on="name",
        suffixes=("_IND", "_WORD"),
    )
    .drop("name", axis=1)
)

In [41]:
merged.minimal_distance.hist(log=True, bins=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
merged.loc[merged.minimal_distance > 50, "geopy_names"] = "unknown"

In [66]:
merged.groupby("geopy_names").size()

geopy_names
Albany                      17
Atka                        28
Bilbao                      62
Binghamton                   4
BuenosAires                 41
Caterpillar                 20
Cortland                    24
Geneseo                     11
Giza                        59
Halifax                     22
Inca_City                   60
Inca_City_Ridges            80
Ithaca                      70
Macclesfield                67
Manhattan2                  22
Manhattan_Classic          100
Manhattan_Cracks            25
Manhattan_Frontinella       37
Oswego_Edge                 41
Pisaq                       20
Portsmouth                  53
Potsdam                     81
Rochester                    9
Schenectady                 23
Starburst                   49
Troy                        32
Wellington                  17
unknown                  61317
dtype: int64

In [43]:
merged.groupby("geopy_names").size()

geopy_names
Albany                    5
Atka                      3
Bilbao                   12
Binghamton                2
BuenosAires              14
Caterpillar               9
Cortland                  1
Geneseo                   1
Giza                     35
Halifax                   3
Inca_City                30
Inca_City_Ridges         33
Ithaca                   50
Macclesfield             35
Manhattan2               11
Manhattan_Classic        45
Manhattan_Cracks         13
Manhattan_Frontinella    21
Oswego_Edge              20
Pisaq                     5
Portsmouth               11
Potsdam                  17
Rochester                 4
Schenectady               9
Starburst                12
Troy                      5
Wellington                2
unknown                  54
dtype: int64

In [67]:
merged = merged.merge(
    meta[["OBSERVATION_START_TIME", "OBSERVATION_ID"]],
    left_on="obsid",
    right_on="OBSERVATION_ID",
).drop("OBSERVATION_ID", axis=1)

In [68]:
merged.rename({"OBSERVATION_START_TIME": "time"}, axis=1, inplace=True)

In [69]:
merged.head(20)

,obsid,lat_IND,lon_IND,geopy_names,minimal_distance,lat_WORD,lon_WORD,time
0,PSP_001330_1395,-40.1045,132.278,unknown,4235.996329,-74.22,168.5,2006-11-08 02:30:23
1,PSP_001340_1945,14.1640,213.001,unknown,10290.947681,-74.22,168.5,2006-11-08 21:28:15
2,PSP_001341_2010,20.8079,184.493,unknown,10605.012103,-74.22,168.5,2006-11-08 23:22:29
3,PSP_001341_2485,68.3594,175.410,unknown,15833.536150,-74.22,168.5,2006-11-08 23:37:32
4,PSP_001341_2650,84.9654,150.396,unknown,17700.908878,-74.22,168.5,2006-11-08 23:42:56
5,PSP_001342_1910,10.7420,158.451,unknown,9454.168657,-74.22,168.5,2006-11-09 01:11:29
6,PSP_001343_1970,16.5792,130.564,unknown,10426.844427,-74.22,168.5,2006-11-09 03:05:25
7,PSP_001343_2510,70.7793,124.003,unknown,16392.458332,-74.22,168.5,2006-11-09 03:22:33
8,PSP_001350_2710,88.9874,188.731,unknown,18135.322657,-74.22,168.5,2006-11-09 16:33:29
9,PSP_001357_2200,39.4768,105.416,unknown,13441.055886,-74.22,168.5,2006-11-10 05:22:16


In [70]:
merged.query("geopy_names == 'Ithaca'")

,obsid,lat_IND,lon_IND,geopy_names,minimal_distance,lat_WORD,lon_WORD,time
61571,PSP_002622_0945,-85.2109,181.474,Ithaca,11.777760,-85.128,180.7,2007-02-16 17:45:48
61572,PSP_002675_0945,-85.2228,181.616,Ithaca,13.643163,-85.128,180.7,2007-02-20 20:52:43
61573,PSP_002820_0945,-85.2078,181.466,Ithaca,11.461588,-85.128,180.7,2007-03-04 04:03:00
61574,PSP_003176_0945,-85.2077,181.478,Ithaca,11.524325,-85.128,180.7,2007-03-31 21:48:06
61575,PSP_003193_0850,-85.1614,180.033,Ithaca,7.326217,-85.128,180.7,2007-04-02 05:33:10
...,...,...,...,...,...,...,...,...
61716,ESP_056699_0950,-85.0537,180.282,Ithaca,9.209898,-85.128,180.7,2018-08-31 11:29:46
61717,ESP_056901_0855,-85.3044,180.898,Ithaca,19.787539,-85.128,180.7,2018-09-16 05:09:04
61718,ESP_056963_0945,-85.3674,176.565,Ithaca,46.662610,-85.128,180.7,2018-09-21 01:06:49
61719,ESP_056976_0950,-85.3124,180.903,Ithaca,20.681368,-85.128,180.7,2018-09-22 01:25:17


In [71]:
from planet4 import stats

In [72]:
stats.define_martian_year(merged, "time")

In [73]:
merged.MY.value_counts()

34    11437
33    10289
31     9772
32     9325
30     8185
29     7877
28     3790
35     1716
Name: MY, dtype: int64

In [74]:
merged.shape

(62391, 9)

In [34]:
merged.to_csv(
    "/Users/klay6683/Dropbox/data/planet4/p4_analysis/obsid_region_names.csv",
    index=False,
)

In [33]:
pd.set_option("display.max_rows", 100)

In [78]:
merged = merged[merged.geopy_names != "unknown"]

In [88]:
merged["MY"].value_counts()

29    176
28    168
31    166
34    159
30    153
32    128
33    124
Name: MY, dtype: int64

In [89]:
merged["MY28"] = merged.MY == 28

In [109]:
hirise_data = merged.groupby(["geopy_names", "MY"]).size().unstack("MY")

In [75]:
to_plot2 = merged.groupby(["geopy_names", "MY"]).size().unstack("MY")

In [117]:
hirise_data.fillna(0).astype("int").to_csv(
    "/Users/klay6683/Dropbox/SternchenAndMe/proposals/2019-11-20-MDAP-P4/all_hirise_data.csv"
)

In [36]:
merged.loc[merged.geopy_names == "unknown"].MY.value_counts()

28    51
29     3
Name: MY, dtype: int64

In [37]:
merged[merged.geopy_names == "unknown"][["lat_IND", "lon_IND", "MY"]]

,lat_IND,lon_IND,MY
14,-84.2131,118.5620,28
16,-84.5605,119.1830,28
19,-84.5618,119.2030,28
26,-84.6815,118.6010,28
29,-84.6916,118.5470,28
30,-84.6947,118.5780,28
31,-84.5610,119.1810,28
33,-84.5473,119.1050,28
35,-84.6301,117.3430,28
48,-82.5135,80.0473,29


In [38]:
to_plot = to_plot[to_plot.MY < 33]

In [39]:
merged[merged.geopy_names == "unknown"][
    ["MY", "obsid", "lat_IND", "lon_IND"]
].to_csv("unknowns.csv", index=False)

In [40]:
to_plot.columns = ["name", "MY", "count"]

In [41]:
to_plot2.head()

MY,28,29,30,31,32
geopy_names,,,,,
Albany,NaN,5.0,NaN,NaN,NaN
Atka,NaN,3.0,NaN,NaN,NaN
Bilbao,2.0,7.0,3.0,NaN,NaN
Binghamton,NaN,2.0,NaN,NaN,NaN
BuenosAires,NaN,7.0,7.0,NaN,NaN


In [42]:
to_plot2 = to_plot2.reset_index()

In [43]:
to_plot2[to_plot2.index=='unknown']

KeyError: False

In [ ]:
to_plot2 = to_plot2[to_plot2.geopy_names!='unknown']

In [ ]:
to_plot2.get(['Caterpillar', 29])

In [ ]:
to_plot2.set_index('geopy_names').at['Caterpillar', 28]

In [ ]:
to_plot2.loc[to_plot2.geopy_names'Albany', 29]

In [ ]:
for name in indexed.index:
    print(name)

In [ ]:
indexed.at['Starburst', 28]

In [ ]:
f, axes = plt.subplots(1, 5, figsize=(6,6), sharey=True,
                       constrained_layout=True)

indexed = to_plot2.set_index('geopy_names')

for yr,ax in zip(range(28,33), axes):
    g = sns.barplot(x=yr, y='geopy_names', ax=ax, data=to_plot2)
    ax.set_ylabel('')
    
#     for name in indexed.index:
#         s = indexed.at[name, yr]
#         ax.text(s, name, s);

In [ ]:
for_table = to_plot2.set_index('geopy_names')

In [ ]:
for_table.index.name="ROI"

In [ ]:
for_table.to_csv("dataset_table.csv")

In [ ]:
merged.shape

In [ ]:
to_plot2 = to_plot2.fillna(0).astype('int')

In [ ]:
sns.countplot(x='MY', y='geopy_names', data=to_plot)

In [ ]:
region_names.set_index('name').drop('point', axis=1).join(for_table).to_csv("dataset_table.csv")

In [ ]:
for_table.join(region_names.set_index('name'))

In [ ]:
merged.time.max()

In [ ]:
to_plot

In [ ]:
!ls *.csv

In [ ]:
status = pd.read_csv("current_status.csv", header=None)

In [ ]:
status.columns = ['obsid', 'done']

In [ ]:
merged.columns

In [ ]:
newmerge = merged.merge(status, on='obsid')

In [ ]:
newmerge[newmerge.done < 25].geopy_names.value_counts()

In [ ]:
import qgrid

# P4 catalog stats

In [47]:
fans = pd.read_csv(
    "/Users/klay6683/Dropbox/data/planet4/p4_analysis/P4_catalog_v1.1/P4_catalog_v1.1_L1C_cut_0.5_fan.csv"
)

In [48]:
fans.head()

,marking_id,angle,distance,tile_id,image_x,image_y,n_votes,obsid,spread,version,...,y_angle,l_s,map_scale,north_azimuth,BodyFixedCoordinateX,BodyFixedCoordinateY,BodyFixedCoordinateZ,PlanetocentricLatitude,PlanetographicLatitude,Longitude
0,F000000,205.56,179.71,APF0000ci9,2270.76,24336.16,35,ESP_012079_0945,88.03,1,...,-0.43,214.785,0.25,126.856883,-65.804336,261.407884,-3370.504345,-85.427383,-85.480829,104.129523
1,F000001,185.39,179.62,APF0000cia,3391.21,5640.60,15,ESP_012079_0945,21.35,1,...,-0.09,214.785,0.25,126.856883,-67.219114,257.011589,-3370.631413,-85.493546,-85.546226,104.656897
2,F000002,184.98,500.27,APF0000cia,3509.96,5876.70,10,ESP_012079_0945,18.91,1,...,-0.09,214.785,0.25,126.856883,-67.170611,257.055226,-3370.630794,-85.493039,-85.545725,104.644396
3,F000004,184.29,105.43,APF0000cia,3716.27,5824.50,6,ESP_012079_0945,26.41,1,...,-0.07,214.785,0.25,126.856883,-67.127761,257.024926,-3370.635002,-85.493723,-85.546401,104.637107
4,F000005,189.42,109.50,APF0000cia,3452.17,6033.00,3,ESP_012079_0945,22.58,1,...,-0.16,214.785,0.25,126.856883,-67.169940,257.096267,-3370.628302,-85.492368,-85.545061,104.642019


In [49]:
fans.groupby("obsid").size().sort_values(ascending=False).head()

obsid
ESP_012316_0925    12304
ESP_012527_0925    10918
ESP_011960_0925     6737
ESP_021520_0925     6585
ESP_021454_0925     6514
dtype: int64

In [50]:
fans.groupby("obsid").size().mean()

737.0976744186047

In [52]:
blotches = pd.read_csv(
    "/Users/klay6683/Dropbox/data/planet4/p4_analysis/P4_catalog_v1.1/P4_catalog_v1.1_L1C_cut_0.5_blotch.csv"
)

In [53]:
blotches.groupby("obsid").size().sort_values(ascending=False).head()

obsid
ESP_012604_0965    20916
ESP_012314_0965    20094
ESP_012248_0965    19264
ESP_012393_0965     9978
ESP_011680_1055     7102
dtype: int64

In [54]:
blotches.groupby("obsid").size().mean()

1130.3212669683257